<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Import</a></span></li></ul></li><li><span><a href="#Data-loading" data-toc-modified-id="Data-loading-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loading</a></span></li><li><span><a href="#Baseline" data-toc-modified-id="Baseline-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Baseline</a></span><ul class="toc-item"><li><span><a href="#Random" data-toc-modified-id="Random-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Random</a></span></li><li><span><a href="#StaticBest" data-toc-modified-id="StaticBest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>StaticBest</a></span><ul class="toc-item"><li><span><a href="#Méthode-1" data-toc-modified-id="Méthode-1-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Méthode 1</a></span></li><li><span><a href="#Méthode-2" data-toc-modified-id="Méthode-2-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Méthode 2</a></span></li></ul></li><li><span><a href="#Optimal" data-toc-modified-id="Optimal-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optimal</a></span></li></ul></li><li><span><a href="#Résolution-par-RL" data-toc-modified-id="Résolution-par-RL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Résolution par RL</a></span><ul class="toc-item"><li><span><a href="#Epsilon-greedy" data-toc-modified-id="Epsilon-greedy-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Epsilon-greedy</a></span></li><li><span><a href="#Upper-Confidence-Bound" data-toc-modified-id="Upper-Confidence-Bound-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Upper-Confidence Bound</a></span></li><li><span><a href="#Lin-UCB" data-toc-modified-id="Lin-UCB-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Lin-UCB</a></span></li></ul></li></ul></div>

Pour ce TP, on va tenter de faire de la publicité ciblée. On a 5 000 articles, 5 thèmes d'articles (chaque article peut avoir un mélange de thèmes), et 10 publicités possibles. Avec différentes méthodes, on cherchera à déterminer la publicité la plus pertinente en général. Enfin dans le dernier article, on cherchera à trouver la publicité la plus pertinente *en fonction* du thème de l'article.

## Import

In [104]:
import random as rd
import numpy as np
from math import log, sqrt
from matplotlib.pyplot import plot

%matplotlib notebook

# Data loading

In [103]:
file1 = open('CTR.txt', 'r') 
Lines = file1.readlines()

In [71]:
Lines[0]

'0:0.74837091377052;0.8352077827766918;0.07669895743095312;0.17243898920810596;0.14344585283799294:0.10341905704918021;0.19069778281037159;0.0;0.10240139196802925;0.0363124254124334;0.07456195450906297;0.2347024151796051;0.0;0.0;0.07857372422425657\n'

In [72]:
data = []
for x in Lines :
    a, b, c = x[:-1].split(":")
    doc_id = np.array(int(a))
    contexte = np.array([float(y) for y in b.split(";")])
    gains = np.array([float(y) for y in c.split(";")])
    data.append([doc_id] + [contexte] + [gains])

In [73]:
num_doc = len(data)
num_doc

5000

In [74]:
data = np.array(data)

/home/jpmerx/.virtualenvs/Jussieu/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [75]:
data[0]

array([array(0),
       array([0.74837091, 0.83520778, 0.07669896, 0.17243899, 0.14344585]),
       array([0.10341906, 0.19069778, 0.        , 0.10240139, 0.03631243,
       0.07456195, 0.23470242, 0.        , 0.        , 0.07857372])],
      dtype=object)

Le tableau ``data`` a 5 000 entrées. Chacune représente un article. La première valeur est le numéro de l'article. La deuxième est une liste d'appartenance de l'article au thème. Enfin la dernière est une liste du taux de clics par publicité à partir de cet article.

# Baseline

## Random

Cette première stratégie est de choisir aléatoirement la publicité affichée sur un article.

In [76]:
def strategy_random(dt):
    size = len(dt)
    regret = [0 for i in range(size + 1)]         #On initialise le regret à 0
    
    for x,i in zip(dt, range(1, size + 1)):       #Pour chaque document : 
        chosen = x[2][rd.randint(0,9)]            #On choisit aléatoirement une publicité
        best = x[2][np.argmax(x[2])]              #On révèle quel aurait été le meilleur choix
        regret[i] = regret[i-1] + best - chosen   #On actualise la valeur du regret
        
    return regret[1:]                             #On retourne le regret moyenné sur le nombre d'échantillons

In [77]:
strategy_random(data)[-1]/5000

0.22899733307706888

In [78]:
x = range(1, num_doc)
y = [a/b for a,b in zip(strategy_random(data), x)]

In [79]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

On atteint avec cette stratégie un regret cumulé de 22.8%.

## StaticBest

### Méthode 1

Cette première stratégie static best consiste à regarder à chaque article *n* la publicité ayant eu le meilleur taux de clics pour les *n-1* articles précédants, et à l'utiliser à ce moment là.

In [80]:
def static_best1(dt):
    size = len(dt)
    regret = [0 for i in range(size + 1)]           #On initialise le regret à 0
    size_bras = len(dt[0][2])
    cumul = [0 for i in range(size_bras)]           #On crée une matrice des cumuls des gains
    
    for x,i in zip(dt, range(1, size + 1)):         #Pour chaque document : 
        chosen = x[2][np.argmax(cumul)]             #On choisit le bras au meilleur gain cumulé
        best = x[2][np.argmax(x[2])]                #On révèle quel aurait été le meilleur choix
        regret[i] = regret[i-1] + best - chosen     #On actualise la valeur du regret
        
        for i in range(size_bras):           
            cumul[i] += x[2][i]                     #On actualise le cumul du gain de chaque bras
            
    return regret[1:]                               #On retourne le regret moyenné sur le nombre d'échantillons

In [81]:
static_best1(data)[-1]/5000

0.03853445621849583

In [82]:
x = range(1, num_doc)
y = [a/b for a,b in zip(static_best1(data), x)]

In [83]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

On obtient un regret cumulé de 3.9%, soit bien mieux qu'avec la stratégie aléatoire.

### Méthode 2

Cette seconde stratégie static best propose de calculer la moyenne du taux de clic de chaque publicité sur tous les articles, et d'utiliser uniquement cette publicité.

In [84]:
def static_best2(dt):
    size = len(dt)
    regret = [0 for i in range(size)]           #On initialise le regret à 0
    size_bras = len(dt[0][2])
    cumul = [0 for i in range(size_bras)]           #On crée une matrice des cumuls des gains
    
    for x in dt:
        for j in range(size_bras):
            cumul[j] = cumul[j] + x[2][j]
    
    average_best = np.argmax(cumul)
    
    for x,k in zip(dt, range(size)):         #Pour chaque document : 
        chosen = x[2][average_best]             #On choisit le bras au meilleur gain cumulé
        best = x[2][np.argmax(x[2])]                #On révèle quel aurait été le meilleur choix
        regret[k] = regret[k-1] + best - chosen     #On actualise la valeur du regret
                 
    return regret[:-1]                               #On retourne le regret moyenné sur le nombre d'échantillons

In [85]:
static_best2(data)[-1]/5000

0.03847364093784723

In [86]:
x = range(1, num_doc)
y = [a/b for a,b in zip(static_best2(data), x)]

In [87]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

On obtient un regret cumulé de 3.8%, soit à peine mieux qu'avec la stratégie précédante.

## Optimal

On fait comme si on savait quelle était la meilleure publicité pour chaque article. Bien sûr, le regret est de 0%.

In [88]:
def optimal(dt):
    size = len(data)
    regret = [0 for i in range(size + 1)]           #On initialise le regret à 0
    
    for x,i in zip(dt, range(1, size + 1)):         #Pour chaque document :
        chosen = x[2][np.argmax(x[2])]              #On choisit le bras au meilleur gain
        best = x[2][np.argmax(x[2])]                #On "révèle" quel aurait été le meilleur choix
        regret[i] = regret[i-1] + best - chosen     #On actualise la valeur du regret
        
    return regret[1:]                               #On retourne le regret moyenné sur le nombre d'échantillons

In [89]:
x = range(1, num_doc)
y = [a/b for a,b in zip(optimal(data), x)]

In [90]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

# Résolution par RL

## Epsilon-greedy

On explore une fois chaque bras. Puis pour $\epsilon$ un paramètre décroissant :

-avec une probabilité $\epsilon$, on choisit un bras au hasard

-avec une probabilité $1-\epsilon$ on choisit le bras qui a eu le meilleur taux de clics jusqu'alors

Cet algorithm ressemble un peu au recuit simulé.

In [91]:
def epsilon_greedy(dt):
    size = len(data)
    regret = [0 for i in range(size)]                                   #On initialise le regret à 0
    
    num_bras = len(dt[0][2])
    cumul = [0 for i in range(num_bras)]                                #On crée une matrice de cumul des gains 
                                                                        #explorés
    num_cumul = [0 for i in range(num_bras)]                            #On crée une matrice du nompre d'exporations de 
                                                                        #chaque bras
        
    for i in range(num_bras):                                           #Tout d'abord, on initialize les cumuls de 
                                                                        #chaque bras :
        cumul[i] = dt[i][2][i]
        num_cumul[i] += 1
        
    for i in range(num_bras, size):                                     #Pour chaque document :
        epsilon = 12.0 / (0.6 * i)                                      #On implémente l'epsilon seuil
        aleat = rd.random()                                             #On simule une variable aléatoire uniforme
        
        if aleat > epsilon:                                             #Dans le cadre 1 - e
            chosen = np.argmax([x/y for x, y in zip(cumul, num_cumul)]) #On choisit le bras le plus fructueux
            
        else:                                                           #Dans le cadre e
            chosen = rd.randint(0, num_bras - 1)                        #On choisit un bras au hasard
            
        cumul[chosen] += dt[i][2][chosen]                               #On ajoute la valeur au cumul
        num_cumul[chosen] += 1
        regret[i] = regret[i-1] +\
                    dt[i][2][np.argmax(dt[i][2])] - dt[i][2][chosen]    #On actualise le regret
        
    return regret[10:]                                                  #On renvoit le regret

In [92]:
epsilon_greedy(data)[-1]/5000

0.042325299025526716

In [93]:
x = range(10, num_doc)
y = [a/b for a,b in zip(epsilon_greedy(data), x)]

In [94]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

On obtient un regret cumulé de 4.7%.

## Upper-Confidence Bound

Il s'agit d'un algorithme optimiste. Au début, on teste une fois chaque bras. Puis, pour chaque bras, on calcule la borne haute de l'intervalle de confiance du taux moyen de clics engendré par la publicité, et on choisit le bras qui a la borne haute la plus élevée. Comment calculer la borne haute ? Moins la publicité a été testée précédemment, plus on ajoute au taux de clics moyen un grand terme (en effet, une moyenne calculée à partir d'un petit nombre d'échantillons est moins exacte). Au taux de clic moyen observé, on ajoute $\sqrt{2 \frac{\ln(n_i)}{n}}$, où $n_i$ est le nombre de fois que le bras a été testé par le passé, et $n$ le nombre de bras testés au total.

In [95]:
def ucb(dt):
    size = len(dt)
    num_bras = len(dt[0][2])
    regret = [0.0 for i in range(size - num_bras + 1)]                  #On initialise le regret à 0
    cumul = [0.0 for i in range(num_bras)]                              #On crée une matrice de cumul des gains 
                                                                        #explorés
    num_cumul = [0.0 for i in range(num_bras)]                          #On crée une matrice du nompre d'exporations de 
                                                                        #chaque bras
        
    for i in range(num_bras):                                           #Tout d'abord, on initialize les cumuls de 
                                                                        #chaque bras :
        cumul[i] = dt[i][2][i]
        num_cumul[i] += 1
        
    for i in range(num_bras, size):                                     #Pour chaque document :
        recompense = [0.0 for i in range(num_bras)]                     #On crée un vecteur reward
        
        for j in range(num_bras):                                       #Pour chaque bras :
            recompense[j] = cumul[j] / num_cumul[j] + \
                            sqrt(2.0 * log(i) / num_cumul[j])           #On calcule la meilleure valeure possible selon
                                                                        #l'intervalle de confiance
        
        chosen = np.argmax(recompense)                                  #On choisit le bras à l'espérance la plus élevée
        cumul[chosen] += dt[i][2][chosen]                               #On ajoute la valeur au cumul
        num_cumul[chosen] += 1
        regret[i - num_bras] = regret[i - 1 - num_bras] + \
                                   dt[i][2][np.argmax(dt[i][2])] - \
                                   dt[i][2][chosen]                     #On actualise le regret
        
    return regret[1:]                                                   #On renvoit le regret

In [96]:
ucb(data)[-2]/5000

0.10674188018595014

In [97]:
x = range(1, num_doc - 10)
y = [a/b for a,b in zip(ucb(data), x)]

In [98]:
%matplotlib notebook
plot(x,y)

<IPython.core.display.Javascript object>

On obtient un regret cumulé de 10.7%.

## Lin-UCB

L'idée est d'utiliser le contexte. Un régression de Ridge est faite pour estimer l'espérance de gain de chaque bras, en fonction des thèmes de l'article. Ensuite, comme pour UCB, une borne haute de cette espérance est utilisée pour choisir la publicité.

In [99]:
def lin_ucb(dt, alpha):
    size = len(dt)
    num_bras = len(dt[0][2])
    num_feature = len(dt[0][1])
    regret = [0.0 for i in range(size + 1)]                            
    
    matrix_a = [np.identity(num_feature) for i in range(num_bras)]
    matrix_b = [np.zeros((num_feature,)) for i in range(num_bras)]
    
    for i in range(size):
        theta = [np.linalg.inv(matrix_a[i]) @ \
                 matrix_b[i] for i in range(num_bras)]
        matrix_p = [np.transpose(theta[i]) @ dt[i][1] + \
                   alpha * sqrt(np.transpose(dt[i][1]) @ \
                   np.linalg.inv(matrix_a[i]) @ dt[i][1]) \
                   for i in range(num_bras)]
        
        chosen = np.argmax(matrix_p)
        regret_actuel = dt[i][2][chosen]
        matrix_a[chosen] = matrix_a[chosen] + dt[i][1] @ \
                           np.transpose(dt[i][1])
        matrix_b[chosen] = regret_actuel * np.array(dt[i][1]) + \
                           matrix_b[chosen]
        regret[i+1] = regret[i] + regret_actuel
    
    return regret[1:]

In [100]:
lin_ucb(data, 0.06)[-1]/5000

0.018394550793498776

In [101]:
x = range(1, num_doc)
y = [[a/b for a,b in zip(lin_ucb(data, 0.01 * (i+1)), x)] for i in range(10)]

In [102]:
colors = ['black', 'red', 'green', 'orange', 'yellow', 'magenta', 'blue', 'grey', 'pink', 'cyan']
for i in range(10):
    plot(x,y[i], color=colors[i])

On obtient un regret cumulé de 1.8%, soit la meilleure technique (appliquable) jusqu'à présent !